# ***Import les librairies nécessaires***

In [ ]:
pip install -U giotto-tda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not cur

In [ ]:
pip install torch sickit-learn

ERROR: Could not find a version that satisfies the requirement sickit-learn (from versions: none)
ERROR: No matching distribution found for sickit-learn


In [ ]:
from gtda.homology import VietorisRipsPersistence

# ***Charger le modèle Darija Bert pour obtenir les embeddings***

In [ ]:
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("/content/drive/MyDrive/finetunedmodel_augdataset")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/finetunedmodel_augdataset")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/finetunedmodel_augdataset'. Use `repo_type` argument if needed.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def getEmbeddingByBert(comments):
    comments_embeddings = []
    for comment in comments:
        inputs = tokenizer(comment, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        comment_emb = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        comments_embeddings.append(comment_emb)
    return np.vstack(comments_embeddings)

In [ ]:
import pandas as pd

ds = pd.read_excel(r'/content/sample_data/XGBoost_training_dataset.xlsx')
comments = ds['Comment'].tolist()

In [ ]:
ds.Polarité.value_counts()

# ***Division de dataset (train-test)***

In [ ]:
X_Emb = getEmbeddingByBert(comments)

In [ ]:
#créer un dictionnaire qui convertit nos émotions en un chiffre pertinent
#1. liste des émotions existants
labels_existants= ds.Polarité.unique()

#2. énumérer les labels
labels ={}
for i,labels_existants in enumerate(labels_existants):
    labels[labels_existants] = i

#3. Ajouter une colonne au dataframe nommé label contenat les nombres uniques
ds['label'] = ds.Polarité.replace(labels)
ds.head()

/tmp/ipython-input-181937676.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ds['label'] = ds.Polarité.replace(labels)


,Comment,Polarité,label
0,مع احترامي المحتوى ديالك والمجهودات المبذولة ح...,NEG,0
1,جوج اسفنج 55درهم,NEG,0
2,العنوان غلط فييييين غادي ينصبو عليك في مراكش...,NEG,0
3,من جهت الفطور المركشي مكاملش اغالي البيصرة وال...,NEG,0
4,ورينا فين نعسو بالرخاء فمراكش,NTR,1


In [ ]:
y = ds.label.values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


#Diviser le dataset pour l'entrainement et test
X_train, X_test, y_train, y_test = train_test_split(X_Emb, y, test_size=0.2, stratify=y, random_state=42)

# ***Ajouter Caractéristiques aux embeddings***

In [ ]:
X_train.shape

(2933, 768)

In [ ]:
from gtda.diagrams import PersistenceImage

vr = VietorisRipsPersistence(homology_dimensions=[0, 1])

In [ ]:
X_train_reshaped = [vec.reshape(-1, 1) for vec in X_train]
diagrams = vr.fit_transform(X_train_reshaped)


In [ ]:
from gtda.diagrams import PersistenceEntropy

pe = PersistenceEntropy()
tda_pe = pe.fit_transform(diagrams)

In [ ]:
tda_pe.shape

(2933, 2)

In [ ]:
X_combined = np.concatenate([X_train, tda_pe], axis=1)

# ***Gradient Boosting Classifier***

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
"""Cet algorithme construit un modèle additif étape par étape ; il permet l'optimisation de fonctions de perte différentiables arbitraires.
 À chaque étape, des arbres de régression de n classes sont ajustés sur le gradient négatif de la fonction de perte"""
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {
    'n_estimators': [100, 200],        # nombre d'arbres, par défaut =100
    'learning_rate': [0.01, 0.1, 0.2], # 0.1 est la valeur par défaut de scikit learn; controler la vitesse de convergence
    'max_depth': [3, 5, 10],           # profondeur d'arbres de décision,elle limite le nombre de nœuds dans l'arbre. par défaut 3
    'subsample': [0.8, 1.0]            # Fraction d'échantillons à utiliser pour ajuster individual base learner, par défaut =1, 0.8 chaque arbre utilise 80% du dataset
}

#choisir les meilleurs hyperparamètres, entrainer modèle, validation croisée avec 3 folds
gb = GradientBoostingClassifier(random_state=42)
gb_grid = GridSearchCV(gb, param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
gb_grid.fit(X_plusfeatures, y_train)

# best parameters
print("Meilleurs paramètres :", gb_grid.best_params_)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

model = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='mlogloss')
score = cross_val_score(model, X_combined, y_train, cv=5, scoring='f1_macro')
print("F1-score (XGBoost avec TDA) :", score.mean())

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:46:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:46:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:46:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:46:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:47:11] WARNING: /w

F1-score (XGBoost avec TDA) : 0.9439766583547133


## ***test***

In [ ]:
import pandas as pd

ds = pd.read_excel(r'/content/sample_data/test.xlsx')
comments = ds['Comment'].tolist()

In [ ]:
y_test2=ds.label.values

In [ ]:
y_test2

array(['NEG', 'NEG', 'NTR', 'NEG', 'NEG', 'NEG', 'NEG', 'NEG', 'NEG',
       'NEG', 'NTR', 'NTR', 'NTR', 'NTR', 'NEG', 'NTR', 'NTR', 'NTR',
       'NEG', 'NTR', 'POS', 'POS', 'NTR', 'NEG', 'POS', 'POS', 'NTR',
       'POS', 'POS'], dtype=object)

In [ ]:
#2. énumérer les labels
y_test2 = [ 0 if x == 'NEG' else 1 if x == 'NTR'else 2 for x in y_test2 ]


In [ ]:
X_test2 = getEmbeddingByBert(comments)

In [ ]:
from gtda.diagrams import PersistenceImage

X_test_reshaped2 = [vec.reshape(-1, 1) for vec in X_test2]
diagrams_test2 = vr.transform(X_test_reshaped2)

In [ ]:
pe_tda_test2 = pe.transform(diagrams_test2)

In [ ]:
Xtest_combined = np.concatenate([X_test2, pe_tda_test2], axis=1)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

model.fit(X_combined, y_train)
y_pred = model.predict(Xtest_combined)

print("Accuracy:", accuracy_score(y_test2, y_pred))
print("F1-score (macro):", f1_score(y_test2, y_pred, average='macro'))

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [22:47:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.896551724137931
F1-score (macro): 0.8964368964368964


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

precision = precision_score(y_test2, y_pred,average='macro')
recall = recall_score(y_test2, y_pred,average='macro')

print(f'Precision: {precision*100:.2f}%')
print(f'Recall: {recall*100:.2f}%')

Precision: 91.90%
Recall: 88.38%
